In [1]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 28.5 MB/s 


In [2]:
import pandas as pd

import regex as re

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

In [3]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('vader_lexicon')
from textblob import TextBlob
from nltk import tokenize

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv", encoding = "ISO-8859-1")

In [5]:
df= df[['Review Text','Recommended IND']].dropna()
df.rename(columns = {'Review Text':'text', 'Recommended IND': 'Recommended'}, inplace = True)
df.columns

Index(['text', 'Recommended'], dtype='object')

In [6]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=False)

,text,Recommended
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


In [7]:
df.isnull().sum()

text           0
Recommended    0
dtype: int64

In [8]:
df.shape

(22641, 2)

In [9]:
def clean_text(tweet):
      tweet = re.sub(r'http\S+', '', tweet)  # remove URLs
      tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
      tweet = re.sub('@[A-Za-z0-9_]+', '', tweet)  # remove hashtags
      tweet = re.sub('#[A-Za-z0-9_]+', '', tweet)  # remove mentions
      tweet = re.sub(r'[^\w\s]', '', tweet)  # remove punctuations
      tweet = re.sub('\s+', ' ', tweet)  # remove extra whitespace
      tweet = re.sub('[ \t]+$', '', tweet)  # remove extra whitespace
      return tweet

In [10]:
df["cleantxt"] = df["text"].apply(clean_text)

In [11]:
df["cleantxt"] = df["cleantxt"].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df.head()

,text,Recommended,cleantxt
0,Absolutely wonderful - silky and sexy and comf...,1,Absolutely wonderful silky sexy comfortable
1,Love this dress! it's sooo pretty. i happene...,1,Love this dress sooo pretty happened find stor...
2,I had such high hopes for this dress and reall...,0,such high hopes this dress really wanted work ...
3,"I love, love, love this jumpsuit. it's fun, fl...",1,love love love this jumpsuit flirty fabulous e...
4,This shirt is very flattering to all due to th...,1,This shirt very flattering adjustable front pe...


In [12]:
tokenized_text = df["cleantxt"].apply(lambda x: x.split())
tokenized_text.head()

0    [Absolutely, wonderful, silky, sexy, comfortable]
1    [Love, this, dress, sooo, pretty, happened, fi...
2    [such, high, hopes, this, dress, really, wante...
3    [love, love, love, this, jumpsuit, flirty, fab...
4    [This, shirt, very, flattering, adjustable, fr...
Name: cleantxt, dtype: object

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
sia = SentimentIntensityAnalyzer()
df['scores'] = df['cleantxt'].apply(lambda text: sia.polarity_scores(text))
df= df[['cleantxt','Recommended', 'scores']].dropna()
df.head(10)

,cleantxt,Recommended,scores
0,Absolutely wonderful silky sexy comfortable,1,"{'neg': 0.0, 'neu': 0.154, 'pos': 0.846, 'comp..."
1,Love this dress sooo pretty happened find stor...,1,"{'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'comp..."
2,such high hopes this dress really wanted work ...,0,"{'neg': 0.043, 'neu': 0.803, 'pos': 0.154, 'co..."
3,love love love this jumpsuit flirty fabulous e...,1,"{'neg': 0.19, 'neu': 0.206, 'pos': 0.604, 'com..."
4,This shirt very flattering adjustable front pe...,1,"{'neg': 0.0, 'neu': 0.544, 'pos': 0.456, 'comp..."
5,love tracy reese dresses this very petite just...,0,"{'neg': 0.0, 'neu': 0.767, 'pos': 0.233, 'comp..."
6,aded this basket last mintue what would look l...,1,"{'neg': 0.047, 'neu': 0.83, 'pos': 0.123, 'com..."
7,ordered this carbon store pick stuff always us...,1,"{'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'comp..."
8,love this dress usually runs little snug bust ...,1,"{'neg': 0.0, 'neu': 0.702, 'pos': 0.298, 'comp..."
9,ordered petite make sure length wasnt long typ...,1,"{'neg': 0.0, 'neu': 0.63, 'pos': 0.37, 'compou..."


In [15]:
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment_type']=''
df.loc[df.compound>0,'sentiment_type']='POSITIVE'
df.loc[df.compound==0,'sentiment_type']='NEUTRAL'
df.loc[df.compound<0,'sentiment_type']='NEGATIVE'
df.tail(10)

,cleantxt,Recommended,scores,compound,sentiment_type
23475,Cute dress waist high sleeves tight maybe diff...,1,"{'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compou...",0.7717,POSITIVE
23476,These bottoms very cute defiantly cheeky would...,1,"{'neg': 0.0, 'neu': 0.585, 'pos': 0.415, 'comp...",0.7264,POSITIVE
23477,impressed with beautiful color combinations em...,1,"{'neg': 0.051, 'neu': 0.835, 'pos': 0.115, 'co...",0.5994,POSITIVE
23478,surprised positive reviews this product terrib...,0,"{'neg': 0.309, 'neu': 0.451, 'pos': 0.24, 'com...",-0.3169,NEGATIVE
23479,wasnt sure about ordering this skirt because c...,1,"{'neg': 0.072, 'neu': 0.716, 'pos': 0.211, 'co...",0.8195,POSITIVE
23481,very happy snag this dress such great price ve...,1,"{'neg': 0.0, 'neu': 0.442, 'pos': 0.558, 'comp...",0.9310,POSITIVE
23482,reminds maternity clothes soft stretchy shiny ...,1,"{'neg': 0.056, 'neu': 0.664, 'pos': 0.28, 'com...",0.7506,POSITIVE
23483,This well very through this never would have w...,0,"{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'comp...",0.8481,POSITIVE
23484,bought this dress wedding have this summer cut...,1,"{'neg': 0.104, 'neu': 0.673, 'pos': 0.223, 'co...",0.7353,POSITIVE
23485,This dress lovely platinum feminine fits perfe...,1,"{'neg': 0.0, 'neu': 0.369, 'pos': 0.631, 'comp...",0.9286,POSITIVE
